### 데이터 불러오기

In [1]:
import pandas as pd

file_paths={
    'international_trade': 'C:/Users/82106/Desktop\제주도 과일 공모전/data/international_trade.csv',
    'test': 'C:/Users/82106/Desktop\제주도 과일 공모전/data/test.csv',
    'train': 'C:/Users/82106/Desktop/제주도 과일 공모전/data/train.csv',
    'sample_submission':'C:/Users/82106/Desktop\제주도 과일 공모전/data/sample_submission.csv'
}

#각 파일을 데이터프레임으로 읽기 
dataframes={name: pd.read_csv(path) for name,path in file_paths.items()}


### 날짜형 변환

In [2]:
dataframes['international_trade']['timestamp']=pd.to_datetime(dataframes['international_trade']['기간'],format='%Y-%m')
train_timestamp_format=dataframes['train']['timestamp'].dtypes
test_timestamp_format=dataframes['test']['timestamp'].dtypes

### 품목명과 item 의 unique 값

In [3]:
unique_trade_items=dataframes['international_trade']['품목명'].unique()
unique_train_items=dataframes['train']['item'].unique()

### 수출중량 , 수입 중량 , 무역수지 group화 -> monthl_trade_stats 로 특성 엔지니어링-> 날짜형 변환

In [4]:
monthly_trade_stats=dataframes['international_trade'].groupby(dataframes['international_trade']['timestamp'].dt.to_period('M')).agg({
    '수출 중량': 'sum',
    '수입 중량': 'sum',
     '무역수지': 'sum'
}).reset_index()

monthly_trade_stats['timestamp']=monthly_trade_stats['timestamp'].dt.to_timestamp()


### train, test 날짜형 변환 

In [5]:
dataframes['train']['timestamp']=pd.to_datetime(dataframes['train']['timestamp'])
dataframes['test']['timestamp']=pd.to_datetime(dataframes['test']['timestamp'])
monthly_trade_stats['timestamp']=pd.to_datetime(monthly_trade_stats['timestamp'])

#병합시도
train_with_stats=dataframes['train'].merge(monthly_trade_stats, on='timestamp',how='left')
train_with_stats_filled=train_with_stats.fillna(method='ffill')
test_with_stats=dataframes['test'].merge(monthly_trade_stats, on='timestamp',how='left')
test_with_stats_filled=test_with_stats.fillna(method='ffill')

### 범주형 변수 원핫인코딩

In [6]:
categorical_columns=['item','corporation','location']
train_encoded=pd.get_dummies(train_with_stats_filled,columns=categorical_columns)
test_encoded=pd.get_dummies(test_with_stats_filled,columns=categorical_columns)

### train data 의 timestamp 컬럼을 연도 월 일 요일로 분해

In [7]:
train_encoded['year']=train_encoded['timestamp'].dt.year
train_encoded['month']=train_encoded['timestamp'].dt.month
train_encoded['day']=train_encoded['timestamp'].dt.day
train_encoded['weekday']=train_encoded['timestamp'].dt.weekday

In [8]:
# 원래의 timestamp column 제거

train_encoded.drop('timestamp',axis=1,inplace=True)

### test data 의 timestamp 컬럼을 연도 월 일 요일로 분해

In [9]:
test_encoded['year']=test_encoded['timestamp'].dt.year
test_encoded['month']=test_encoded['timestamp'].dt.month
test_encoded['day']=test_encoded['timestamp'].dt.day
test_encoded['weekday']=test_encoded['timestamp'].dt.weekday

In [10]:
#테스트데이터도 timestamp 컬럼 제거

test_encoded.drop('timestamp',axis=1,inplace=True)

### train_encdoed 데이터 학습 과 검정 세트로 분할

In [11]:
from sklearn.model_selection import train_test_split

X=train_encoded.drop(columns=['price(원/kg)','ID','supply(kg)'],errors='ignore')
y=train_encoded['price(원/kg)']
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.2,random_state=42)

### XGBOOST 회귀 파라미터값 설정, 학습, 예측 

In [12]:
from xgboost import XGBRegressor
model=XGBRegressor(
n_estimators=10,
max_depth=10,
learning_rate=0.1,
colsample_bytree=0.8,
subsample=0.8,
objective='reg:squarederror',
random_state=42)

In [13]:
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
y_valid_pred=model.predict(X_valid)

### 검증세트에 대한 예측 수행


In [15]:
from sklearn.metrics import r2_score

# R^2 score 계산
r2_valid = r2_score(y_valid, y_valid_pred)

r2_valid

0.6488964637066957

### 그리드 서치

### 그리드 서치 파라미터 범위 설정

In [16]:
from sklearn.model_selection import GridSearchCV
# 모델 초기화
xgb_model = XGBRegressor(random_state=42)

param_grid = {
    'n_estimators': [80, 150],  # 주어진 모델보다 낮은 값을 포함하여 비교
    'max_depth': [5, 7],         # 주어진 모델의 값과 비슷하거나 더 높은 값을 포함
    'learning_rate': [0.05, 0.15],  # 학습률을 조금 더 높여 볼 수 있음
    'colsample_bytree': [0.6, 1.0],  # 모든 컬럼을 사용하는 경우 포함
    'subsample': [0.5, 1.0]         # 행의 샘플링 비율도 조정
}


### 그리드 서치 객체 초기화

In [17]:
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,                # 3-fold cross-validation
    scoring='r2',        # R^2 score를 기준으로 최적화
    verbose=1,           # 진행 상황에 대한 상세 출력
    n_jobs=-1            # 모든 CPU 코어 사용
)

# 학습 데이터 준비
X = train_encoded.drop(columns=['price(원/kg)', 'ID', 'supply(kg)'], errors='ignore')
y = train_encoded['price(원/kg)']

# 그리드 서치 실행
grid_search.fit(X, y)

# 결과 출력: 최적의 파라미터와 해당할 때의 R^2 스코어
best_parameters = grid_search.best_params_


best_parameters

Fitting 3 folds for each of 32 candidates, totalling 96 fits


{'colsample_bytree': 1.0,
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 80,
 'subsample': 0.5}

### 하이퍼 파라미터로 학습

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 최적의 하이퍼파라미터로 모델을 정의
model = XGBRegressor(
    n_estimators=best_parameters['n_estimators'],
    max_depth=best_parameters['max_depth'],
    learning_rate=best_parameters['learning_rate'],
    colsample_bytree=best_parameters['colsample_bytree'],
    subsample=best_parameters['subsample'],
    objective='reg:squarederror',
    random_state=42
)

# 모델을 학습 세트에 학습
model.fit(X_train, y_train)

# 검증 세트에 대한 예측을 수행
y_valid_pred = model.predict(X_valid)

# R^2 스코어로 모델을 평가)
r2 = r2_score(y_valid, y_valid_pred)

r2


0.745589674049661